In [1]:
from keras import applications, optimizers, utils
from keras import backend as K
from keras import models as Models
from keras.datasets import cifar, cifar10
from keras.models import Model 
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import os
import sys
import time

img_width, img_height = 32, 32
train_data_dir = "train_set"
validation_data_dir = "test_set"
batch_size = 32
epochs = 10

Using TensorFlow backend.


In [2]:
model = applications.VGG16(weights = "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5", include_top=False, input_shape = (img_width, img_height, 3))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [3]:
# Freeze layers
for layer in model.layers:
   layer.trainable = False

# Create output
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(10, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

print(model_final.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [4]:
def cifar10_reduced_load_data(data_path, target_width, target_height):
    """Loads CIFAR10 dataset.

    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """
    num_train_samples = 50000
    h_factor = target_height / 32
    w_factor = target_width / 32

    x_train = np.empty((num_train_samples, 3, 32, 32), dtype='uint8')
    y_train = np.empty((num_train_samples,), dtype='uint8')

    for i in range(1, 6):
        fpath = os.path.join(data_path, 'data_batch_' + str(i))
        (x_train[(i - 1) * 10000: i * 10000, :, :, :],
         y_train[(i - 1) * 10000: i * 10000]) = cifar.load_batch(fpath)

    fpath = os.path.join(data_path, 'test_batch')
    x_test, y_test = cifar.load_batch(fpath)

    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))

    if K.image_data_format() == 'channels_last':
        x_train = x_train.transpose(0, 2, 3, 1)
        x_test = x_test.transpose(0, 2, 3, 1)
        
    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = cifar10_reduced_load_data("cifar-10-batches-py", 256, 256)

y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

print("X Train %s - X Test %s" % (x_train.shape, x_test.shape, ) )
print("Y Train %s - Y Test %s" % (y_train.shape, y_test.shape, ) )


X Train (50000, 32, 32, 3) - X Test (10000, 32, 32, 3)
Y Train (50000, 10) - Y Test (10000, 10)


In [5]:
# Train the model
time_start = time.time()

model_final.fit(
    x_train,
    y_train,
    epochs = epochs,
    validation_data = (x_test, y_test))

time_elapsed = time.time() - time_start

# Record time used
print(time_elapsed)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 30s 590us/step - loss: 3.8619 - acc: 0.3956 - val_loss: 1.5029 - val_acc: 0.5249
Epoch 2/10
50000/50000 [==============================] - 28s 560us/step - loss: 1.7877 - acc: 0.4691 - val_loss: 1.3377 - val_acc: 0.5537
Epoch 3/10
50000/50000 [==============================] - 28s 564us/step - loss: 1.5350 - acc: 0.5015 - val_loss: 1.2893 - val_acc: 0.5666
Epoch 4/10
50000/50000 [==============================] - 29s 572us/step - loss: 1.4131 - acc: 0.5246 - val_loss: 1.2571 - val_acc: 0.5750
Epoch 5/10
50000/50000 [==============================] - 29s 572us/step - loss: 1.3483 - acc: 0.5430 - val_loss: 1.2281 - val_acc: 0.5836
Epoch 6/10
50000/50000 [==============================] - 29s 575us/step - loss: 1.2969 - acc: 0.5558 - val_loss: 1.2110 - val_acc: 0.5899
Epoch 7/10
50000/50000 [==============================] - 29s 574us/step - loss: 1.2644 - acc: 0.5660 - val_loss: 1.

In [7]:
score = model_final.evaluate(x_test, y_test)
print('Test score:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 4s 405us/step
Test score: 1.16237779465
Test accuracy: 0.6049
